<a href="https://colab.research.google.com/github/angelynafaa/DATAMINING/blob/main/Pattern_FPGrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
upload = files.upload()

Saving movie.csv to movie.csv


In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
data=pd.read_csv('movie.csv')

In [ ]:
data.fillna(value=0,axis=1,inplace=True)
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,0,Doug Walker,0.0,0.0,131.0,0.0,Rob Walker,131.0,0.0,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,8,143,0,0.0,0,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,0.0,0,0,0,0.0,0.0,12.0,7.1,0.00,0


In [ ]:
data.drop(['duration', 'actor_3_facebook_likes', 'actor_2_facebook_likes', 'actor_1_facebook_likes', 'movie_imdb_link' , 'country', 'movie_facebook_likes', 'budget', 'title_year'], axis=1, inplace=True)

In [ ]:
data.drop(['aspect_ratio'], axis=1, inplace=True)

In [ ]:
data.drop(['color', 'num_critic_for_reviews', 'director_facebook_likes', 'gross', 'num_voted_users' , 'cast_total_facebook_likes', 'facenumber_in_poster', 'plot_keywords', 'num_user_for_reviews', 'language', 'content_rating'],axis=1,inplace=True)

In [ ]:
data.head()

,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,imdb_score
0,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,7.9
1,Gore Verbinski,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,7.1
2,Sam Mendes,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,6.8
3,Christopher Nolan,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,8.5
4,Doug Walker,Rob Walker,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,0,7.1


In [ ]:
data.shape

(5043, 7)

In [ ]:
DatasetMovie=[]
for x in range(0,1000):
    DatasetMovie.append([str(data.values[x,y]) for y in range(0,7)])

In [ ]:
DatasetMovie[:1]

[['James Cameron',
  'Joel David Moore',
  'Action|Adventure|Fantasy|Sci-Fi',
  'CCH Pounder',
  'Avatar\xa0',
  'Wes Studi',
  '7.9']]

In [ ]:
def getReccomendation(r):
    for x in range(0,1000):
      if (DatasetMovie)[x][3] == r[3] or (DatasetMovie)[x][2] == r[0] or (DatasetMovie)[x][2] == r[2]:
        if DatasetMovie[x][4] != r[4]:
          return DatasetMovie[x]
        else:
          return random.choice(DatasetMovie)
      else: 
        return random.choice(DatasetMovie)

In [ ]:
movieRiwayat=[]
userRiwayat=[]
for x in range(0,1000):
    firtsMovie = (random.choice(DatasetMovie))
    userRiwayat.append([])
    movieRiwayat.append([])
    movieRiwayat[x].append(getReccomendation(firtsMovie))
    for y in range(0,10):
      movieRiwayat[x].append(getReccomendation(movieRiwayat[x][y]))
      userRiwayat[x].append(movieRiwayat[x][y][4].replace(u'\xa0', u''))

In [ ]:
userRiwayat[:1]

[['Bullet to the Head',
  'Monsters University',
  'Up',
  'After the Sunset',
  'Terminator Genisys',
  'Any Given Sunday',
  'The Lord of the Rings: The Fellowship of the Ring',
  'Anger Management',
  'Me, Myself & Irene',
  'Shaft']]

In [ ]:
pip install pyfpgrowth

     |████████████████████████████████| 1.6MB 5.4MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5479 sha256=ee703365fa67e655144957cccf93190083c6c2c38eb2874438ee7f6f77b9370a
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [ ]:
import pyfpgrowth

In [ ]:
transactions = userRiwayat
sup = 0.003  # float(input("Enter support %: "))
patterns = pyfpgrowth.find_frequent_patterns(transactions, int(len(transactions) * sup))
print(len(patterns))
rules = pyfpgrowth.generate_association_rules(patterns, 0.2)
print(rules)
print(len(rules))

In [ ]:
rekomendasi=[]
for x,y in rules.items():
  rekom=[]
  rekom.append(x)
  rekom.append(y[0])
  rekom.append(y[1])

  rekomendasi.append(rekom)
  rules_df=pd.DataFrame(rekomendasi,columns=['Movie','Rekomendasi','Confidence'])
rules_df

,Movie,Rekomendasi,Confidence
0,"(The Amazing Spider-Man 2,)","(Avatar,)",0.800000
1,"(Epic,)","(Harry Potter and the Chamber of Secrets,)",0.375000
2,"(Harry Potter and the Chamber of Secrets,)","(Epic,)",0.500000
3,"(Noah,)","(The Siege,)",0.214286
4,"(What Women Want,)","(Noah,)",0.500000
...,...,...,...
82,"(Crimson Tide,)","(The Emperor's New Groove,)",0.214286
83,"(Now You See Me,)","(The Legend of Tarzan,)",0.200000
84,"(Elizabethtown,)","(Mission: Impossible - Ghost Protocol,)",0.200000
85,"(The Simpsons Movie,)","(Avatar,)",0.200000


In [ ]:
df=pd.DataFrame(userRiwayat)
df['Items'] = df[df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df.drop(df.columns[0:6],axis=1)

def support_count(sup):  
    count=0
    sup1=set(str(sup))
    sup1.remove("'")
    sup1.remove('(')
    sup1.remove(')')
    sup1.discard(' ')
    for j in df['Items']:
        j=set(j)
        if(sup1.issubset(j)):
            count=count+1
    return count
support=[]
for i in rules_df['Rekomendasi']:
    a=support_count(i)
    support.append(a/len(df))
rules_df['support']=support

In [ ]:
rules_df['lift']=rules_df['Confidence']/rules_df['support']

In [ ]:
rules_df[(rules_df['lift'] >= 1.2) &
(rules_df['Confidence'] >= 0.8) & (rules_df['Confidence'] >=0.3)]
rules_df

,Movie,Rekomendasi,Confidence,RHS_support,lift,Support,support
0,"(The Amazing Spider-Man 2,)","(Avatar,)",0.800000,0.505,1.584158,0.505,0.505
1,"(Epic,)","(Harry Potter and the Chamber of Secrets,)",0.375000,0.093,4.032258,0.093,0.093
2,"(Harry Potter and the Chamber of Secrets,)","(Epic,)",0.500000,0.415,1.204819,0.415,0.415
3,"(Noah,)","(The Siege,)",0.214286,0.728,0.294349,0.728,0.728
4,"(What Women Want,)","(Noah,)",0.500000,0.379,1.319261,0.379,0.379
...,...,...,...,...,...,...,...
82,"(Crimson Tide,)","(The Emperor's New Groove,)",0.214286,0.000,inf,0.000,0.000
83,"(Now You See Me,)","(The Legend of Tarzan,)",0.200000,0.100,2.000000,0.100,0.100
84,"(Elizabethtown,)","(Mission: Impossible - Ghost Protocol,)",0.200000,0.043,4.651163,0.043,0.043
85,"(The Simpsons Movie,)","(Avatar,)",0.200000,0.505,0.396040,0.505,0.505


In [ ]:
rules_df.to_csv('FPGROWTH')